In [ ]:
import h5py
from skbeam.core.image import gaussfill
import numpy as np
from pyCXD.detector.eiger import EigerImages
from skimage.restoration import inpaint

from matplotlib.pyplot import figure, ion, clf, imshow, clim, xlim,ylim, subplot
%matplotlib inline


In [ ]:
# load some example data
npdata = np.load("data/imagefill-data.npz")

#this image was averaged over 10 frames
img = npdata['img']
mask = npdata['mask']
x0 = npdata['XCEN']
y0 = npdata['YCEN']

In [ ]:
#img = np.random.random((10,10))
#mask = np.ones_like(img)
#mask[5,6] = 0
#mask[9,4] = 0

In [ ]:
#img_inpaint = inpaint.inpaint_biharmonic(img,mask==0)

In [ ]:
# Navg is 10 because we need to sample from a Poisson distribution assuming
# the count rate was 10 times higher and variance reduced by a factor of sqrt(10)
img_gaussfilled = gaussfill(img,mask,sigma=30,poisson=True,Navg=10)
# calculate the new mask to see what has been filled
newmask_gaussfilled = gaussfill(np.ones(mask.shape), mask, sigma=30) > 1e-6
# we can use a circular average as well

In [ ]:
figure(0);clf()
imshow(mask*img);clim(0,10)
figure(1);clf()
imshow(img_gaussfilled);clim(0,10)
#xlim(1000,1500);ylim(1500,1000)
figure(2);clf();
imshow(img_inpaint)
clim(0,10)


In [ ]:
figure(3);clf();
imshow(newmask_gaussfilled)